<a href="https://colab.research.google.com/github/Likhita-17/4.1_exp/blob/main/imdb_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:

import pandas as pd

messages = pd.read_csv('/content/imdb_labelled.txt', sep='\t',
                           names=["message", "label"])

In [17]:
messages


,message,label
0,Text,Label
1,"A very, very, very slow-moving, aimless movie ...",0
2,Not sure who was more lost - the flat characte...,0
3,Attempting artiness with black & white and cle...,0
4,Very little music or anything to speak of.,0
...,...,...
744,I just got bored watching Jessice Lange take h...,0
745,"Unfortunately, any virtue in this film's produ...",0
746,"In a word, it is embarrassing.",0
747,Exceptionally bad!,0


In [18]:
messages.shape

(749, 2)

In [19]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
PS=PorterStemmer()

In [21]:
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z0-9]',' ',messages['message'][i]) # Keep spaces, remove only non-alphabetic
  review=review.lower()
  review=review.split()
  review=[PS.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review) # Join with space
  corpus.append(review)

In [22]:
corpus

['text',
 'slow move aimless movi distress drift young man',
 'sure lost flat charact audienc nearli half walk',
 'attempt arti black white clever camera angl movi disappoint becam even ridicul act poor plot line almost non exist',
 'littl music anyth speak',
 'best scene movi gerardo tri find song keep run head',
 'rest movi lack art charm mean empti work guess empti',
 'wast two hour',
 'saw movi today thought good effort good messag kid',
 'bit predict',
 'love cast jimmi buffet scienc teacher',
 'babi owl ador',
 'movi show lot florida best made look appeal',
 'song best muppet hilari',
 'cool',
 'right case movi deliv everyth almost right face',
 'averag act main person low budget clearli see',
 'review long overdu sinc consid tale two sister singl greatest film ever made',
 'put gem movi term screenplay cinematographi act post product edit direct aspect film make',
 'practic perfect true masterpiec sea faux masterpiec',
 'structur film easili tightli construct histori cinema 1 th

In [23]:
print(len(corpus))
print(corpus[:5])


749
['text', 'slow move aimless movi distress drift young man', 'sure lost flat charact audienc nearli half walk', 'attempt arti black white clever camera angl movi disappoint becam even ridicul act poor plot line almost non exist', 'littl music anyth speak']


In [24]:
y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values

In [29]:
### TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer(max_features=2500,ngram_range=(1,3)) # Include unigrams and bigrams
X1=cv.fit_transform(corpus).toarray()

In [30]:
from sklearn.model_selection import train_test_split
X1_train,X1_test,y_train,y_test=train_test_split(X1,y,test_size=0.2,random_state=42)


In [31]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(X1_train,y_train)
y1_pred=spam_detect_model.predict(X1_test)

In [32]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print(accuracy_score(y_test,y1_pred))
print(confusion_matrix(y_test,y1_pred))

0.78
[[57 18]
 [15 60]]


In [33]:
print(classification_report(y_test,y1_pred))

              precision    recall  f1-score   support

       False       0.79      0.76      0.78        75
        True       0.77      0.80      0.78        75

    accuracy                           0.78       150
   macro avg       0.78      0.78      0.78       150
weighted avg       0.78      0.78      0.78       150



In [41]:
from sklearn.svm import LinearSVC
svm = LinearSVC(C=1.0, random_state=42)
svm.fit(X1_train, y_train)
y_pred_svm = svm.predict(X1_test)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
print("Classification Report:\n", classification_report(y_test, y_pred_svm))
data=pd.DataFrame({'Actual':y_test,'Predicted':y_pred_svm})
data.to_csv('imdb_predictions.csv',index='false')

SVM Accuracy: 0.8
Confusion Matrix:
 [[59 16]
 [14 61]]
Classification Report:
               precision    recall  f1-score   support

       False       0.81      0.79      0.80        75
        True       0.79      0.81      0.80        75

    accuracy                           0.80       150
   macro avg       0.80      0.80      0.80       150
weighted avg       0.80      0.80      0.80       150



In [43]:
data

,Actual,Predicted
0,False,True
1,False,False
2,False,True
3,True,True
4,True,True
...,...,...
145,True,True
146,False,False
147,False,False
148,True,True


In [39]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='entropy', max_depth=1000, random_state=42)
dt.fit(X1_train, y_train)
y_pred_dt = dt.predict(X1_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_dt))

Decision Tree Accuracy: 0.7066666666666667
Confusion Matrix:
 [[59 16]
 [28 47]]
Classification Report:
               precision    recall  f1-score   support

       False       0.68      0.79      0.73        75
        True       0.75      0.63      0.68        75

    accuracy                           0.71       150
   macro avg       0.71      0.71      0.70       150
weighted avg       0.71      0.71      0.70       150

